In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

# Driver Recording

A CaseRecorder is commonly attached to the problem’s Driver in order to gain insight into the convergence of the model as the driver finds a solution. By default, a recorder attached to a driver will record the design variables, constraints and objectives.

The driver recorder is capable of capturing any values from any part of the model, not just the design variables, constraints, and objectives.

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.core.driver.Driver", recording_options=True)

```{note}
Note that the `excludes` option takes precedence over the `includes` option.
```

## Driver Recording Example

In the example below, we first run a case while recording at the driver level. Then, we examine the objective, constraint, and design variable values at the last recorded case. Lastly, we print the full contents of the last case, including outputs from the problem that are not design variables, constraints, or objectives. 

Specifically, `y1` and `y2` are some of those intermediate outputs that are recorded due to the use of:

`driver.recording_options['includes'] = ['*']`

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src89", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src89`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

import numpy as np

model = SellarDerivatives()

model.nonlinear_solver = om.NonlinearBlockGS()
model.linear_solver = om.ScipyKrylov()

model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9)

driver.recording_options['includes'] = ['*']
driver.recording_options['record_objectives'] = True
driver.recording_options['record_constraints'] = True
driver.recording_options['record_desvars'] = True
driver.recording_options['record_inputs'] = True
driver.recording_options['record_outputs'] = True
driver.recording_options['record_residuals'] = True

driver.add_recorder(om.SqliteRecorder("cases.sql"))

prob = om.Problem(model, driver)
prob.setup()
prob.run_driver()

In [ ]:
prob.cleanup()
cr = om.CaseReader("cases.sql")
driver_cases = cr.list_cases('driver')

In [ ]:
assert len(driver_cases) == 7

In [ ]:
last_case = cr.get_case(driver_cases[-1])
print(last_case)

In [ ]:
last_case.get_objectives()

In [ ]:
assert_near_equal(last_case.get_objectives()['obj'], 3.18339395, tolerance=1e-8)

In [ ]:
last_case.get_design_vars()

In [ ]:
design_vars = last_case.get_design_vars()
assert_near_equal(design_vars['x'], 0., tolerance=1e-8)
assert_near_equal(design_vars['z'][0], 1.97763888, tolerance=1e-8)

In [ ]:
last_case.get_constraints()

In [ ]:
constraints = last_case.get_constraints()
assert_near_equal(constraints['con1'], 0, tolerance=1e-8)
assert_near_equal(constraints['con2'], -20.24472223, tolerance=1e-8)

In [ ]:
last_case.inputs['obj_cmp.x']

In [ ]:
assert_near_equal(last_case.inputs['obj_cmp.x'], 0, tolerance=1e-8)

In [ ]:
last_case.outputs['z']

In [ ]:
assert_near_equal(last_case.outputs['z'][0], 1.97763888, tolerance=1e-8)
assert_near_equal(last_case.outputs['z'][1], 0, tolerance=1e-8)

In [ ]:
last_case.residuals['obj']

In [ ]:
assert_near_equal(last_case.residuals['obj'], 0, tolerance=1e-8)

In [ ]:
last_case['y1']

In [ ]:
assert_near_equal(last_case['y1'], 3.16, tolerance=1e-8)